In [1]:
import logging
import os
import threadpoolctl
import torch
from pathlib import Path
from lightning_fabric import seed_everything

import numpy as np
import pandas as pd
from rectools import Columns

from implicit.als import AlternatingLeastSquares

from rectools.dataset import Dataset
from rectools.metrics import MAP, calc_metrics, MeanInvUserFreq, Serendipity
from rectools.models import ImplicitALSWrapperModel
from rectools.models import SASRecModel
from rectools.models.nn.item_net import CatFeaturesItemNet, IdEmbeddingsItemNet

In [2]:
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"

# For implicit ALS
os.environ["OPENBLAS_NUM_THREADS"] = "1"
threadpoolctl.threadpool_limits(1, "blas")

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)
logger = logging.getLogger()

# Data

In [3]:
# %%time
# !wget -q https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O data_original.zip
# !unzip -o data_original.zip
# !rm data_original.zip

In [4]:
DATA_PATH = Path("data_original")

interactions = (
    pd.read_csv(DATA_PATH / 'interactions.csv', parse_dates=["last_watch_dt"])
    .rename(columns={"last_watch_dt": "datetime"})
)

# Split dataset

In [5]:
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)

# Split to train / test
max_date = interactions[Columns.Datetime].max()
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()
train.drop(train.query("total_dur < 300").index, inplace=True)

# drop items with less than 20 interactions in train
items = train["item_id"].value_counts()
items = items[items >= 20]
items = items.index.to_list()
train = train[train["item_id"].isin(items)]
    
# drop users with less than 2 interactions in train
users = train["user_id"].value_counts()
users = users[users >= 2]
users = users.index.to_list()
train = train[(train["user_id"].isin(users))]

users = train["user_id"].drop_duplicates().to_list()

# drop cold users from test
test_users_sasrec = test[Columns.User].unique()
cold_users = set(test[Columns.User]) - set(train[Columns.User])
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)
test_users = test[Columns.User].unique()


In [6]:
items = pd.read_csv(DATA_PATH / 'items.csv')

In [7]:
# Process item features to the form of a flatten dataframe
items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()
items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
genre_feature = items[["item_id", "genre"]].explode("genre")
genre_feature.columns = ["id", "value"]
genre_feature["feature"] = "genre"
content_feature = items.reindex(columns=[Columns.Item, "content_type"])
content_feature.columns = ["id", "value"]
content_feature["feature"] = "content_type"
item_features = pd.concat((genre_feature, content_feature))

candidate_items = interactions['item_id'].drop_duplicates().astype(int)
test["user_id"] = test["user_id"].astype(int)
test["item_id"] = test["item_id"].astype(int)

catalog=train[Columns.Item].unique()

In [8]:
dataset_no_features = Dataset.construct(
    interactions_df=train,
)

dataset_item_features = Dataset.construct(
    interactions_df=train,
    item_features_df=item_features,
    cat_item_features=["genre", "content_type"],
)

In [9]:
metrics_name = {
    'MAP': MAP,
    'MIUF': MeanInvUserFreq,
    'Serendipity': Serendipity
    

}
metrics = {}
for metric_name, metric in metrics_name.items():
    for k in (1, 5, 10):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

# list with metrics results of all models
features_results = []

# SASRec

In [10]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

## Softmax loss

In [11]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
    recommend_device="cuda",
)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [12]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 4min 48s, sys: 6.03 s, total: 4min 54s
Wall time: 4min 44s


In [13]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/dmtikhono1/git_project/RecTools/rectools/models/nn/transformer_data_preparator.py:250: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/dmtikhono1/git_project/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 26.5 s, sys: 4.86 s, total: 31.4 s
Wall time: 20.5 s


In [14]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "softmax"
features_results.append(metric_values)


In [15]:
features_results

[{'MAP@1': 0.04896729054820606,
  'MAP@5': 0.08284725776567772,
  'MAP@10': 0.09202214080523476,
  'MIUF@1': 3.6266752335962402,
  'MIUF@5': 4.305321882464318,
  'MIUF@10': 4.9095189017166145,
  'Serendipity@1': 0.0009307664840571675,
  'Serendipity@5': 0.0007294075536337563,
  'Serendipity@10': 0.0006691606986540919,
  'model': 'softmax'}]

## BCE loss

In [16]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

In [17]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    loss="BCE",
    item_net_block_types=(IdEmbeddingsItemNet, )  # Use only item ids in ItemNetBlock
)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [18]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 4min 53s, sys: 6.57 s, total: 5min
Wall time: 4min 51s


In [19]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn",
)

/data/home/dmtikhono1/git_project/RecTools/rectools/models/nn/transformer_data_preparator.py:250: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/dmtikhono1/git_project/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 26.5 s, sys: 4.68 s, total: 31.1 s
Wall time: 20.5 s


In [20]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "bce"
features_results.append(metric_values)

## gBCE loss

In [21]:
RANDOM_SEED = 32
torch.use_deterministic_algorithms(True)
seed_everything(RANDOM_SEED, workers=True)

Seed set to 32


32

In [22]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    loss="gBCE",
    n_negatives=256,
    gbce_t=0.75,
    item_net_block_types=(IdEmbeddingsItemNet, )  # Use only item ids in ItemNetBlock
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [23]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 1h 59min 3s, sys: 27.5 s, total: 1h 59min 30s
Wall time: 9min 25s


In [24]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)


/data/home/dmtikhono1/git_project/RecTools/rectools/models/nn/transformer_data_preparator.py:250: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/dmtikhono1/git_project/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 28.6 s, sys: 5.07 s, total: 33.7 s
Wall time: 20.7 s


In [25]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "gBCE"
features_results.append(metric_values)

## Softmax with key_padding_mask + causal_mask

In [26]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, ),  # Use only item ids in ItemNetBlock
    recommend_device="cuda",
    use_key_padding_mask=True,
)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [27]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 4min 49s, sys: 8 s, total: 4min 57s
Wall time: 4min 46s


In [28]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/dmtikhono1/git_project/RecTools/rectools/models/nn/transformer_data_preparator.py:250: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/dmtikhono1/git_project/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 26.2 s, sys: 4.67 s, total: 30.9 s
Wall time: 20.2 s


In [29]:
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)

In [30]:
metric_values["model"] = "softmax_padding_mask"
features_results.append(metric_values)

In [31]:
features_df = (
    pd.DataFrame(features_results)
    .set_index("model")
    .sort_values(by=["MAP@10", "Serendipity@10"], ascending=False)
)
features_df

,MAP@1,MAP@5,MAP@10,MIUF@1,MIUF@5,MIUF@10,Serendipity@1,Serendipity@5,Serendipity@10
model,,,,,,,,,
softmax,0.048967,0.082847,0.092022,3.626675,4.305322,4.909519,0.000931,0.000729,0.000669
softmax_padding_mask,0.046881,0.080974,0.090133,3.661533,4.169721,4.791547,0.000869,0.000667,0.000614
gBCE,0.047546,0.081190,0.089990,3.389033,3.982107,4.670658,0.000666,0.000544,0.000521
bce,0.043528,0.074286,0.083131,3.677649,4.437024,4.988751,0.000480,0.000509,0.000531


In [32]:
torch.cuda.empty_cache()

### sasrec with item ids embeddings in ItemNetBlock

In [33]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, )  # Use only item ids in ItemNetBlock
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [34]:
%%time
model.fit(dataset_no_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


CPU times: user 4min 54s, sys: 7.35 s, total: 5min 1s
Wall time: 4min 51s


In [35]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_no_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/dmtikhono1/git_project/RecTools/rectools/models/nn/transformer_data_preparator.py:250: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/dmtikhono1/git_project/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 28.1 s, sys: 4.99 s, total: 33.1 s
Wall time: 21.5 s


In [36]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_ids"
features_results.append(metric_values)

### sasrec with item ids and category features embeddings in ItemNetBlock

model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(IdEmbeddingsItemNet, CatFeaturesItemNet)  # Use item ids and cat features in ItemNetBlock
)

In [39]:
#%%time
model.fit(dataset_item_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 927 K 
---------------------------------------------------------------
927 K     Trainable params
0         Non-trainable params
927 K     Total params
3.709     Total estimated model params size (MB)
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


In [40]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/dmtikhono1/git_project/RecTools/rectools/models/nn/transformer_data_preparator.py:250: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/dmtikhono1/git_project/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 29.4 s, sys: 5.36 s, total: 34.8 s
Wall time: 23.5 s


In [41]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_ids_cat"
features_results.append(metric_values)

### sasrec with category item features embeddings in ItemNetBlock

In [42]:
model = SASRecModel(
    n_blocks=2,
    session_max_len=32,
    lr=1e-3,
    epochs=5,
    verbose=1,
    deterministic=True,
    item_net_block_types=(CatFeaturesItemNet, )  # Use only cat item features in ItemNetBlock
)

Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [43]:
#%%time
model.fit(dataset_item_features)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name        | Type                           | Params
---------------------------------------------------------------
0 | torch_model | TransformerBasedSessionEncoder | 211 K 
---------------------------------------------------------------
211 K     Trainable params
0         Non-trainable params
211 K     Total params
0.847     Total estimated model params size (MB)
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=143` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

/data/home/dmtikhono1/git_project/RecTools/rectools/dataset/features.py:424: UserWarning: Converting sparse features to dense array may cause MemoryError
  warnings.warn("Converting sparse features to dense array may cause MemoryError")
`Trainer.fit` stopped: `max_epochs=5` reached.


In [44]:
%%time
recos = model.recommend(
    users=test_users_sasrec, 
    dataset=dataset_item_features,
    k=10,
    filter_viewed=True,
    on_unsupported_targets="warn"
)

/data/home/dmtikhono1/git_project/RecTools/rectools/models/nn/transformer_data_preparator.py:250: UserWarning: 91202 target users were considered cold because of missing known items
  warnings.warn(explanation)
/data/home/dmtikhono1/git_project/RecTools/rectools/models/base.py:675: UserWarning: 
                Model `<class 'rectools.models.nn.sasrec.SASRecModel'>` doesn't support recommendations for cold users,
                but some of given users are cold: they are not in the `dataset.user_id_map`
            
  warnings.warn(explanation)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/data/home/dmtikhono1/git_project/RecTools/.venv/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the 

Predicting: |          | 0/? [00:00<?, ?it/s]

CPU times: user 28.3 s, sys: 4.93 s, total: 33.2 s
Wall time: 22.5 s


In [45]:
# TODO: drop `apply(str)`
recos["item_id"] = recos["item_id"].apply(str)
test["item_id"] = test["item_id"].astype(str)
metric_values = calc_metrics(metrics, recos[["user_id", "item_id", "rank"]], test, train, catalog)
metric_values["model"] = "sasrec_cat"
features_results.append(metric_values)

In [50]:
features_results

[{'MAP@1': 0.04896729054820606,
  'MAP@5': 0.08284725776567772,
  'MAP@10': 0.09202214080523476,
  'MIUF@1': 3.6266752335962402,
  'MIUF@5': 4.305321882464318,
  'MIUF@10': 4.9095189017166145,
  'Serendipity@1': 0.0009307664840571675,
  'Serendipity@5': 0.0007294075536337563,
  'Serendipity@10': 0.0006691606986540919,
  'model': 'softmax'},
 {'MAP@1': 0.043528276756084665,
  'MAP@5': 0.07428604222209606,
  'MAP@10': 0.08313070132284942,
  'MIUF@1': 3.677649254022451,
  'MIUF@5': 4.437024188083364,
  'MIUF@10': 4.988751384673628,
  'Serendipity@1': 0.00047959964836482004,
  'Serendipity@5': 0.0005090243033398202,
  'Serendipity@10': 0.0005313065730108525,
  'model': 'bce'},
 {'MAP@1': 0.047545855054294456,
  'MAP@5': 0.0811899212734244,
  'MAP@10': 0.08998959207113556,
  'MIUF@1': 3.3890332750515273,
  'MIUF@5': 3.982106805298122,
  'MIUF@10': 4.670657694445981,
  'Serendipity@1': 0.000665940853529614,
  'Serendipity@5': 0.0005438011709959203,
  'Serendipity@10': 0.0005214735119313822,


### Item to item

In [46]:
target_items = [13865, 4457, 15297]

In [47]:
%%time
recos = model.recommend_to_items(
    target_items=target_items, 
    dataset=dataset_no_features,
    k=10,
    filter_itself=True,
    items_to_recommend=None, #white_list,
)

CPU times: user 1.35 s, sys: 272 ms, total: 1.62 s
Wall time: 1.1 s


In [48]:
recos

,target_item_id,item_id,score,rank
0,13865,15648,1.000000,1
1,13865,3386,1.000000,2
2,13865,147,0.892528,3
3,13865,16194,0.892528,4
4,13865,12309,0.892528,5
5,13865,12586,0.892528,6
6,13865,6661,0.892528,7
7,13865,2255,0.892528,8
8,13865,3792,0.892528,9
9,13865,4130,0.892528,10


In [52]:
# TODO: change model for recos (here is the last one trained and is is the worst in quality)
recos.merge(items[["item_id", "title"]], on="item_id")

,target_item_id,item_id,score,rank,title
0,13865,15648,1.000000,1,Черное золото
1,13865,3386,1.000000,2,Спартак
2,13865,147,0.892528,3,Единичка
3,13865,16194,0.892528,4,Голубая линия
4,13865,12309,0.892528,5,Враг у ворот
5,13865,12586,0.892528,6,Вспоминая 1942
6,13865,6661,0.892528,7,Солдатик
7,13865,2255,0.892528,8,Пленный
8,13865,3792,0.892528,9,Собибор
9,13865,4130,0.892528,10,Пустота


In [49]:
torch.cuda.empty_cache()